In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import pandas as pd
from extract_feats import *
from hydrate_tweets import *

In [3]:
df = pd.read_csv('data/training.csv')
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


In [4]:
label_mapping = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear'}

In [5]:
'''
features = {'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'joy': 4, 'sadness': 5,
            'surprise': 6, 'trust': 7, 'negative': 8, 'positive': 9, 'CC': 10, 'IN': 11, 'JJR': 12,
            'JJS': 13, 'PRP': 14, '!': 15, '?': 16}
nrc_pos_feats = extract_nrc_pos_feats(df, features)
nrc_pos_feats
'''

"\nfeatures = {'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'joy': 4, 'sadness': 5,\n            'surprise': 6, 'trust': 7, 'negative': 8, 'positive': 9, 'CC': 10, 'IN': 11, 'JJR': 12,\n            'JJS': 13, 'PRP': 14, '!': 15, '?': 16}\nnrc_pos_feats = extract_nrc_pos_feats(df, features)\nnrc_pos_feats\n"

In [6]:
cleaned_data = cleanData(df)
cleaned_data

,text,label
0,didnt feel humili,0
1,go feel hopeless damn hope around someon care ...,0
2,im grab minut post feel greedi wrong,3
3,ever feel nostalg fireplac know still properti,2
4,feel grouchi,3
...,...,...
15995,veri brief time beanbag said anna feel like be...,0
15996,turn feel pathet still wait tabl sub teach degre,0
15997,feel strong good overal,1
15998,feel like thi wa rude comment im glad,3


In [7]:
X, y, vectorizer = createBoWFeatureVec(cleaned_data)

In [8]:
X, y

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([0, 0, 3, ..., 1, 3, 0]))

In [9]:
X2, y, vectorizer2 = createTfIdfFeatureVec(cleaned_data)

In [10]:
X2, y

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 3, ..., 1, 3, 0]))

In [11]:
X_train, X_test, y_train, y_test = splitData(X,y)

KeyboardInterrupt: 

In [ ]:
X_train2, X_test2, y_train2, y_test2 = splitData(X2,y)

In [ ]:
clf, accuracy = trainLogisticRegression(X_train, X_test, y_train, y_test)

In [ ]:
clf2, accuracy2 = trainLogisticRegression(X_train2, X_test2, y_train2, y_test2)

In [ ]:
accuracy 'using BoW'

0.853

In [ ]:
accuracy2 'using TfIdF'

0.817

In [37]:
df = pd.read_csv('data/corona_tweets/corona_tweets_01.csv', header=None)
df

,0,1
0,1240727808080412673,0.357143
1,1240727808005079041,0.116071
2,1240727808340414464,-0.050000
3,1240727808629813248,-0.714286
4,1240727808617230336,0.700000
...,...,...
831322,1240861453524930562,0.000000
831323,1240861453554315265,0.140693
831324,1240861453503959043,0.000000
831325,1240861453311070209,0.000000


In [38]:
tweet_fields=['created_at'],
user_fields=['description']

text_data, author_data, timestamp_data = hydrateTweets(df, tweet_fields, user_fields)
d = {'text': text_data, 'author': author_data, 'timestamp': timestamp_data}
tweet_data = pd.DataFrame(data=d)
tweet_data

,text,author,timestamp
0,longer defend make excuse reasoning want inten...,trying use wits good evil people united change...,2020-03-19T19:52:15.000Z
1,everyone looks sick,starchild go wild :dizzy: :butterfly: ig remin...,2020-03-19T19:52:15.000Z
2,good came china,,2020-03-19T19:52:15.000Z
3,corona day feels like sunday ...,,2020-03-19T19:52:15.000Z
4,close president notes seen crossed corona repl...,,2020-03-19T19:52:15.000Z
...,...,...,...
56,asked dc cop noticed since coronavirus sent pe...,ut-business marketing mastersmgmt aquinas prou...,2020-03-19T19:52:18.000Z
57,lot would actually benefit taking good look in...,,2020-03-19T19:52:18.000Z
58,close president notes seen crossed corona repl...,i'm great believer luck find harder work thoma...,2020-03-19T19:52:18.000Z
59,among things shows they're using term coronavi...,pro market pro immigrant pro gmo anti supersti...,2020-03-19T19:52:18.000Z


In [39]:
X, y = createBoWFeatureVecTweets(tweet_data['text'].tolist(), df[1].values, vectorizer)
X, y

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0]]),
 array([ 0.35714286,  0.11607143, -0.05      , ...,  0.        ,
         0.        ,  0.        ]))

In [40]:
X, y = createTfIdfFeatureVecTweets(tweet_data['text'].tolist(), df[1].values, vectorizer2)
X, y

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.19842294,
         0.        ]]),
 array([ 0.35714286,  0.11607143, -0.05      , ...,  0.        ,
         0.        ,  0.        ]))